In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
# detect and init the TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()

    # instantiate a distribution strategy
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.TPUStrategy(tpu)
except:
    pass
%pip install glob
%pip install nibabel

In [ ]:

import tarfile
file = tarfile.open('/kaggle/input/BraTS2021_Training_Data.tar')

file.extractall('./BraTS2021_Training_Data')
file.close()
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all messages are logged (default behavior)
                                          # 1 = INFO messages are not printed
                                          # 2 = INFO and WARNING messages are not printed
                                          # 3 = INFO, WARNING, and ERROR messages are not printed
import tensorflow as tf
tf.config.run_functions_eagerly(False)
tf.get_logger().setLevel('ERROR')


In [ ]:
import tensorflow as tf
import tensorflow.keras as kr
import tensorflow.keras as keras
import nibabel as nib
import numpy as np
import os
from glob import glob
import matplotlib.pylab as plt
import keras
import random
import time



folder_path = [os.path.join(r"BraTS2021_Training_Data", i) for i in os.listdir(r"BraTS2021_Training_Data")]
imgs = []


for i in folder_path:
    imgs.append(glob(i+"/*"))
print("Done")
folder_path = folder_path[1:]
imgs = imgs[1:]
print(imgs[0])



#input directory and output directory
outputpath = []
inputpath = []
for j in imgs:
    for i in j:
        if "seg.nii.gz" in i.split("_"):
            outputpath.append(i)
for j in imgs:
    temppath = []
    for i in j:
        if "seg.nii.gz" not in i.split("_"):
            temppath.append(i)
    inputpath.append(temppath)
    
x_train_path = inputpath[:1126].copy()
y_train_path = outputpath[:1126].copy()
x_eval_path = inputpath[1126:1189].copy()
y_eval_path = outputpath[1126:1189].copy()
x_test_path = inputpath[1189:].copy()
y_test_path = outputpath[1189:].copy()  

parition = {"train": x_train_path, "validation": x_eval_path}
label = {"train": y_train_path, "validation": y_eval_path}



#set up var
x_VolSlice_start = 10
x_VolSlice_end = 150

y_VolSlice_start = 10
y_VolSlice_end = 150


#-----------------------------------------------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------model making--------------------------------------------------------------------------------



class GradientAccumulationModel(tf.keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n_gradients = tf.Variable(0, dtype=tf.int32, trainable=False)
        self.n_acum_steps = tf.Variable(4, dtype=tf.int32, trainable=False)
        self.gradient_accumulation = [tf.Variable(tf.zeros_like(v, dtype=tf.float32), trainable=False) for v in self.trainable_variables]

    def train_step(self, data):
        self.n_gradients.assign_add(1)
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        
        gradients = tape.gradient(loss, self.trainable_variables)
        
        for i in range(len(self.gradient_accumulation)):
            self.gradient_accumulation[i].assign_add(gradients[i])
        
        tf.cond(tf.equal(self.n_gradients, self.n_acum_steps), 
                lambda: self.apply_accu_gradients(), 
                lambda: tf.no_op())
        
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}

    def apply_accu_gradients(self):
        self.optimizer.apply_gradients(zip(self.gradient_accumulation, self.trainable_variables))
        
        for i in range(len(self.gradient_accumulation)):
            self.gradient_accumulation[i].assign(tf.zeros_like(self.trainable_variables[i], dtype=tf.float32))
        
        self.n_gradients.assign(0)

#note:
# conv3d need a 5d tensor (height,width,depth,channels)


import tensorflow as tf
from tensorflow.keras import layers, Model
# dice loss as defined above for 4 classes
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)


 
# These functions are used for evaluating the performance of a segmentation model on three different classes
# in medical imaging (presumably related to brain tumor segmentation).
# Input Parameters:
# y_true: The ground truth segmentation mask for the edema class.
# y_pred: The predicted segmentation mask for the edema class.
# epsilon: A small constant to avoid division by zero.


import tensorflow as tf
from tensorflow import keras
K = keras.backend

def dice_coef(y_true, y_pred, smooth=1.):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Update other metric functions similarly
def dice_coef_necrotic(y_true, y_pred, epsilon=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = K.sum(K.abs(y_true[:,:,:,1] * y_pred[:,:,:,1]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,1])) + K.sum(K.square(y_pred[:,:,:,1])) + epsilon)

def dice_coef_edema(y_true, y_pred, epsilon=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = K.sum(K.abs(y_true[:,:,:,2] * y_pred[:,:,:,2]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,2])) + K.sum(K.square(y_pred[:,:,:,2])) + epsilon)

def dice_coef_enhancing(y_true, y_pred, epsilon=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = K.sum(K.abs(y_true[:,:,:,3] * y_pred[:,:,:,3]))
    return (2. * intersection) / (K.sum(K.square(y_true[:,:,:,3])) + K.sum(K.square(y_pred[:,:,:,3])) + epsilon)

# Update other metric functions if necessary
def precision(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def sensitivity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())


from tensorflow.keras import layers

def conv_block_3d(input_tensor, num_filters):
    x = layers.Conv3D(num_filters, 3, padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Dropout(0.3)(x)  # Add dropout
    x = layers.Conv3D(num_filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def unet_3d(input_shape=(156, 156, 140, 4), num_classes=4):
    inputs = layers.Input(input_shape, dtype='float16')
    
    # Encoder (downsampling)
    conv1 = conv_block_3d(inputs, 32)
    pool1 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    
    conv2 = conv_block_3d(pool1, 64)
    pool2 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    
    conv3 = conv_block_3d(pool2, 128)
    pool3 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    
    conv4 = conv_block_3d(pool3, 256)
    pool4 = layers.MaxPooling3D(pool_size=(2, 2, 2))(conv4)
    
    # Bridge
    conv5 = conv_block_3d(pool4, 512)
    
    # Decoder (upsampling)
    up6 = layers.Conv3DTranspose(256, 2, strides=(2, 2, 2), padding='same')(conv5)
    up6 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv4.shape[1] - x.shape[1]], [0, conv4.shape[2] - x.shape[2]], [0, conv4.shape[3] - x.shape[3]], [0, 0]]))(up6)
    concat6 = layers.Concatenate()([up6, conv4])
    conv6 = conv_block_3d(concat6, 256)
    
    up7 = layers.Conv3DTranspose(128, 2, strides=(2, 2, 2), padding='same')(conv6)
    up7 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv3.shape[1] - x.shape[1]], [0, conv3.shape[2] - x.shape[2]], [0, conv3.shape[3] - x.shape[3]], [0, 0]]))(up7)
    concat7 = layers.Concatenate()([up7, conv3])
    conv7 = conv_block_3d(concat7, 128)
    
    up8 = layers.Conv3DTranspose(64, 2, strides=(2, 2, 2), padding='same')(conv7)
    up8 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv2.shape[1] - x.shape[1]], [0, conv2.shape[2] - x.shape[2]], [0, conv2.shape[3] - x.shape[3]], [0, 0]]))(up8)
    concat8 = layers.Concatenate()([up8, conv2])
    conv8 = conv_block_3d(concat8, 64)
    
    up9 = layers.Conv3DTranspose(32, 2, strides=(2, 2, 2), padding='same')(conv8)
    up9 = layers.Lambda(lambda x: tf.pad(x, [[0, 0], [0, conv1.shape[1] - x.shape[1]], [0, conv1.shape[2] - x.shape[2]], [0, conv1.shape[3] - x.shape[3]], [0, 0]]))(up9)
    concat9 = layers.Concatenate()([up9, conv1])
    conv9 = conv_block_3d(concat9, 32)
    
    outputs = layers.Conv3D(num_classes, 1, activation='softmax', dtype='float32')(conv9)
    
    model = GradientAccumulationModel(inputs=inputs, outputs=outputs)
    return model

import tensorflow as tf
tf.keras.backend.clear_session()
def lr_schedule(epoch):
    initial_lr = 0.001
    drop = 0.5
    epochs_drop = 5.0
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

# Add lr_scheduler to your callbacks list when fitting the model
print("start")


def weighted_categorical_crossentropy(class_weights):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        weighted_losses = y_true * class_weights * tf.math.log(y_pred + 1e-7)
        return -tf.reduce_sum(weighted_losses, -1)
    return loss
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=(1,2,3))
    denominator = tf.reduce_sum(y_true + y_pred, axis=(1,2,3))
    return 1 - numerator / denominator

def combined_loss(y_true, y_pred):
    class_weights = tf.constant([0.1, 1.0, 1.0, 1.0])
    wce = weighted_categorical_crossentropy(class_weights)(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return wce + dice
# Example class weights (adjust based on your data)
class_weights = tf.constant([0.1, 1.0, 1.0, 1.0])

try:
    with tpu_strategy.scope():
        model = unet_3d(input_shape=(156, 156, 140, 4), num_classes=4)
        model.compile(loss=combined_loss,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=4),
                       dice_coef, precision, sensitivity, specificity,
                       dice_coef_necrotic, dice_coef_edema, dice_coef_enhancing])
    tf.debugging.set_log_device_placement(True)
except:
    model = unet_3d(input_shape=(156, 156, 140, 4), num_classes=4)
    model.compile(loss=weighted_categorical_crossentropy(class_weights),
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy', tf.keras.metrics.MeanIoU(num_classes=4),
                       dice_coef, precision, sensitivity, specificity,
                       dice_coef_necrotic, dice_coef_edema, dice_coef_enhancing])
    tf.debugging.set_log_device_placement(True)
    
model.summary()



def augment(image, mask):
    # Random flip left-right
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        mask = tf.image.flip_left_right(mask)
    
    # Random flip up-down
    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_up_down(image)
        mask = tf.image.flip_up_down(mask)
    
    # Random rotation
    angle = tf.random.uniform((), minval=-0.1, maxval=0.1)
    image = tf.contrib.image.rotate(image, angle)
    mask = tf.contrib.image.rotate(mask, angle)
    
    return image, mask

# In your __data_generation method, after loading and preprocessing:


#datagen
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=5, dim=(156,156,140), n_channels=4,
                 n_classes=10, shuffle=True, real_batchsize_custom=2, frames_chunk=18,
                 y_VolSlice_start=0, y_VolSlice_end=None):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.real_batchsize_custom = real_batchsize_custom
        self.frames_chunk = frames_chunk
        self.y_VolSlice_start = y_VolSlice_start
        self.y_VolSlice_end = y_VolSlice_end if y_VolSlice_end is not None else dim[2]

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        sliced_dim = (self.dim[0], self.dim[1], self.y_VolSlice_end - self.y_VolSlice_start)
        X = np.empty((self.real_batchsize_custom, *sliced_dim, 4))
        y = np.empty((self.real_batchsize_custom,*sliced_dim, 4), dtype=float)

        initial_point = 0
        final_point = self.frames_chunk
        for x in range(self.real_batchsize_custom):    
            indexes_orig = indexes[initial_point:final_point]
            list_IDs_temp = [self.list_IDs[k] for k in indexes_orig]
            X_in, y_in = self.__data_generation(list_IDs_temp)
            X[x] = X_in
            y[x] = y_in
            initial_point = final_point
            final_point = final_point + self.frames_chunk   
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        sliced_dim = (self.dim[0], self.dim[1], 140)
        X = np.empty((1, *sliced_dim, 4))
        y = np.zeros((1,*sliced_dim,4))
        
        random.seed(time.time())

        for i, file_paths in enumerate(list_IDs_temp):
            image = self.load_nifti(file_paths)
                
            X[0] = image
            
            # Set the label
            seg = nib.load(self.labels[i]).get_fdata().astype(np.float16)[:,:,self.y_VolSlice_start:self.y_VolSlice_end]
            seg[seg==4] = 3

            re= np.expand_dims(seg, axis=0)

            re = tf.image.resize(re,(156,156))
            re = re[0]
            seg = tf.cast(re, tf.int32)
            print(i)
            mask = tf.one_hot(seg, 4)
            
            y[0] = mask
            
        X = X.astype(np.float16)
        y = y.astype(np.float16)
        
        # Improved normalization
        X_norm = np.zeros_like(X[0])
        for c in range(X[0].shape[-1]):  # Iterate over channels
            channel = X[0][..., c]
            min_val = np.min(channel)
            max_val = np.max(channel)
            if max_val > min_val:
                X_norm[..., c] = (channel - min_val) / (max_val - min_val)
            else:
                X_norm[..., c] = channel  # If max == min, keep original values
        X_norm, y[0] = self.augment(X_norm, y[0])
        return X_norm, y[0]

    def load_nifti(self, file_paths):
        i = 0
        temp = nib.load(inputpath[i][0]).get_fdata().astype(np.float16)[:,:,self.y_VolSlice_start:self.y_VolSlice_end]
        re= np.expand_dims(temp, axis=0)
        re = tf.image.resize(re,(156,156))
        temp = re[0]
        
        temp = np.expand_dims(temp, axis=3)
        for channels in range(1,len(file_paths)):
            tempo = nib.load(file_paths[channels]).get_fdata().astype(np.float16)[:,:,self.y_VolSlice_start:self.y_VolSlice_end]
            re= np.expand_dims(tempo, axis=0)
            re = tf.image.resize(re,(156,156))
            tempo = re[0]
            tempo = np.expand_dims(tempo, axis=3)
            temp = np.concatenate((temp,tempo),axis=3)
        
        return temp
training_generator = DataGenerator(parition["train"],label["train"])
valid_generator = DataGenerator(parition["validation"],label["validation"])

#model fitting


from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

filepath="3D-UNet-2018-weights-improvement-{epoch:02d}-{val_accuracy:.3f}.keras" 

checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

csv_logger = CSVLogger('training_2021_2D_UNet.log')

history =  model.fit(training_generator,
                    epochs=12,
                    steps_per_epoch=len(parition["train"]),
                    callbacks= [checkpoint, csv_logger, early_stop, lr_scheduler],
                    validation_data = valid_generator
                    )

with open('history.txt', 'w') as f:
    f.write(history)
try:
    model.save("final_model.h5")
except:
    model.save("final_model.h5")

model

In [ ]:
model